In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

In [ ]:
MOCKUP = False

In [ ]:
from automl.rl.environment.gymnasium_env import GymnasiumEnvironmentWrapper

# Choose environment to test

environment_to_test = (GymnasiumEnvironmentWrapper, {})
#environment_to_test = None

In [ ]:
import basic_dqn_hp_experiment as experiment
#import basic_ppo_hp_experiment as experiment

In [ ]:
from automl.meta_rl.hyperparameter_suggestion import HyperparameterSuggestion

hyperparameters_to_change : list[HyperparameterSuggestion] = experiment.get_hyperparameters_to_change()

In [ ]:
configuration_dict = experiment.get_configuration_dict(num_episodes=300, env=environment_to_test, mockup=MOCKUP)

In [ ]:
from automl.rl.evaluators.rl_std_avg_evaluator import LastValuesAvgStdEvaluator
from automl.rl.evaluators.rl_evaluator_player import EvaluatorWithPlayer


#evaluator = None
#evaluator = (LastValuesAvgStdEvaluator, {"value_to_use" : "episode_steps"})
evaluator = (EvaluatorWithPlayer, {"base_evaluator" : (LastValuesAvgStdEvaluator, {"value_to_use" : "episode_steps"})})

In [ ]:
hp_optimization_input = experiment.gen_hp_optimization_input(hyperparameters_to_change, configuration_dict, evaluator=evaluator)

In [ ]:
from automl.base_configurations.base_configurations import load_configuration_dict

hp_optimization_input["n_trials"] = 5
hp_optimization_input["steps"] = 5

In [ ]:
from automl.meta_rl.hp_optimization_pipeline import HyperparameterOptimizationPipeline

hp_optimization_pipeline = HyperparameterOptimizationPipeline(hp_optimization_input)


In [ ]:
hp_optimization_pipeline.setup_files() # partially initialize the hp optimization pipeline with the current input, it can be manually changed before it is used

In [ ]:
experiment_directory = hp_optimization_pipeline.get_artifact_directory()

In [ ]:
from automl.basic_components.state_management import load_component_from_folder

hp_optimization_pipeline : HyperparameterOptimizationPipeline = load_component_from_folder(experiment_directory)

In [ ]:
#import torch
#torch.autograd.set_detect_anomaly(True) # use this in case of certain types of errors

hp_optimization_pipeline.run()

In [ ]:
experiment_path = hp_optimization_pipeline.get_artifact_directory()

print(experiment_path)

In [ ]:
from automl.basic_components.state_management import save_state

save_state(hp_optimization_pipeline)